#### Download data 

We will use [weather data](https://www.bgc-jena.mpg.de/wetter/) provided by [Max Plank Institute for BiogeoChemistry](https://www.bgc-jena.mpg.de).

In [ ]:
#Download the data
!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip --quiet

In [ ]:
!ls -l

In [ ]:
#Unzip the file
!unzip jena_climate_2009_2016.csv.zip

In [ ]:
!ls -l

#### Data Exploration

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
#Read dataset
df = pd.read_csv('jena_climate_2009_2016.csv')

In [ ]:
#Let's check the dataset contents
df.head(n=5)

In [ ]:
#Columns
df.columns

In [ ]:
df.tail()

In [ ]:
#Check number of records
df.shape

What is the Frequency of this Time-Series data?

In [ ]:
df.head(n=10)

In [ ]:
#Check if null values
df.isnull().sum()

How should we deal missing values in Time Series data?

#### Data for Air Tempreture

Excercise #1 : Build a model which will predict future 'Air Tempreture' based on the past data.

In [ ]:
#Get data for Air Tempreture
temp_df = df['T (degC)']

In [ ]:
temp_df

In [ ]:
#Make Date time column as index to make sure data is sorted
temp_df.index = df['Date Time']

In [ ]:
#Check data
temp_df.head(n=50)

In [ ]:
temp_df.tail(n=50)

In [ ]:
#Visualize data
temp_df.plot(subplots=True, figsize=(10,6))
plt.show()

In [ ]:
temp_df.shape

#### Data Preparation

Split data between Training and Test

In [ ]:
#Number of training examples - set to 80%
num_training_examples = int(0.8 * temp_df.shape[0])
num_training_examples

In [ ]:
#Prepare training and test data
train_data = temp_df.to_list()[:num_training_examples]
test_data = temp_df.to_list()[num_training_examples:]

In [ ]:
#Check the data
print(train_data[:100])

In [ ]:
#Find the mean standard deviation
mean = np.array(train_data).mean()
std = np.array(train_data).std()

In [ ]:
mean, std

Normalize the data

In [ ]:
#Normalize training and test data
norm_train = (np.array(train_data) - mean)/std
norm_test  = (np.array(test_data) - mean)/std

In [ ]:
norm_train[:100]

In [ ]:
def prepare_xy(dataset, window_size=20):

    dataX, dataY = [], []
    
    for i in range(len(dataset)-window_size):
        
        #Prepare input features
        input_features = dataset[i:(i+window_size)]        
        dataX.append(input_features)

        #Prepare Label
        label = dataset[i + window_size]
        dataY.append(label)
    
    return np.array(dataX), np.array(dataY)

In [ ]:
#Prepare x,y for train and test
train_x, train_y = prepare_xy(norm_train, window_size=20)
test_x, test_y = prepare_xy(norm_test, window_size=20)

Build X (Input features) and y (Label) for Model training

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
#First example input features (x)
train_x[0]

In [ ]:
#First example label (y)
train_y[0]

Visualize single example

In [ ]:
def show_example(input_features, target, prediction=None):

    #Plot historical values
    plt.plot(list(range(input_features.shape[0])), input_features.flatten(), '.-', label='History' )
    
    #Plot target
    plt.plot(input_features.shape[0]+1, target, 'rx', markersize=10, label='Actual')

    #Plot prediction, if applicable
    if prediction:
        plt.plot(input_features.shape[0]+1, prediction, 'go', markersize=10, label='Prediction')

    plt.legend()
    plt.show()

In [ ]:
#Display examples
exp_num = np.random.randint(0, train_x.shape[0])
show_example(train_x[exp_num], train_y[exp_num])

Prediction using Average method

In [ ]:
#Prediction is taken as average of all the points in sequence e.g 20
exp_num = np.random.randint(0, train_x.shape[0])
show_example(train_x[exp_num], train_y[exp_num], train_x[exp_num].mean())

#### Build Model

In [ ]:
import tensorflow as tf

In [ ]:
#Random seed for reproducibility
tf.random.set_seed(13)

In [ ]:
#Build Model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

#Add LSTM layer
model.add(tf.keras.layers.LSTM(8, input_shape=(20, 1)))

In [ ]:
model.output

In [ ]:
#Add Output layer
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.output

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mae')

In [ ]:
model.summary()

Train the model

- Convert batch to be 3 dimensional data : Batch size x Sequence length x 1

In [ ]:
model.fit(train_x, train_y,
          validation_data=(test_x, test_y),
          epochs=10, 
          batch_size=256)

In [ ]:
train_x.shape

In [ ]:
model.input

In [ ]:
train_x = np.expand_dims(train_x, axis=2)
test_x = np.expand_dims(test_x, axis=2)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
model.fit(train_x, train_y,
          validation_data=(test_x, test_y),
          epochs=10, 
          batch_size=256)

In [ ]:
test_x.shape

In [ ]:
#Model Prediction on first example
a = model.predict(test_x[0:1])
print('Normalized Prediction', a)

In [ ]:
print('De-normalized Prediction', a * std + mean)

In [ ]:
#Actual 
test_y[0]*std+mean

Visualize Model prediction

In [ ]:
test_x[0].shape

In [ ]:
model.input

In [ ]:
np.expand_dims(test_x[1], axis=0).shape

In [ ]:
#Pick a test example
exp_num = np.random.randint(0, test_x.shape[0])

#Make input example a batch of 1
prediction = model.predict(np.expand_dims(test_x[exp_num], axis=0))

#Visualize
show_example(test_x[exp_num], test_y[exp_num], prediction[0])

#### Mutiple Time Series - Data Preparation

In [ ]:
#Air tempreture, pressure and air density
features_to_include = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']

#Get data
multi_df = df[features_to_include]
multi_df.index = df['Date Time']

In [ ]:
multi_df.head(n=10)

In [ ]:
num_training_examples = int(0.8 * multi_df.shape[0])

In [ ]:
#Train and Test data
train_df = multi_df.iloc[:num_training_examples,:]
test_df = multi_df.iloc[num_training_examples:,:]

In [ ]:
train_df.shape

Normalize data

In [ ]:
#Find mean and standard deviation
mean_multi = train_df.mean()
std_multi = train_df.std()

In [ ]:
mean_multi

In [ ]:
std_multi

In [ ]:
#Normalize Train and Test data
norm_train_df = (train_df - mean_multi)/std_multi
norm_test_df = (test_df - mean_multi)/std_multi

In [ ]:
norm_train_df.sample(n=5)

Prepare X and Y

In [ ]:
def prepare_xy_multi(dataset, num_time_series=3, window_size=[20,15,10], target_series=0):

    dataX, dataY = [], []

    act_data = []

    for i in range(num_time_series):
        dataX.append([]) #Initialize an empty list for each time series
        act_data.append(dataset.iloc[:,i].to_list())

    #Get max window size
    max_window_size = max(window_size)

    for i in range(len(dataset)-max_window_size):
        
        #Prepare input for each time series
        for j in range(num_time_series):

            #Prepare input features
            input_features = act_data[j][(i+max_window_size-window_size[j]):(i+max_window_size)]
            dataX[j].append(input_features)

        #Prepare Label
        label = act_data[target_series][i + max_window_size]
        dataY.append(label)
    
    return dataX, dataY

In [ ]:
#Prepare Training and Test X, y
train_x_multi, train_y_multi = prepare_xy_multi(norm_train_df)
test_x_multi, test_y_multi = prepare_xy_multi(norm_test_df)

In [ ]:
len(train_x_multi)

In [ ]:
train_x_multi[0][0]

In [ ]:
train_x_multi[1][0]

In [ ]:
train_x_multi[2][0]

#### Build Model II

In [ ]:
import tensorflow as tf

In [ ]:
#Build 3 input layers - one for each time series
input_1 = tf.keras.layers.Input(shape=(20,1)) #Tempreture
input_2 = tf.keras.layers.Input(shape=(15,1)) #Pressure
input_3 = tf.keras.layers.Input(shape=(10,1)) #Relative humidity

In [ ]:
#Build 3 LSTM Layers - One for each time series
lstm_1 = tf.keras.layers.LSTM(8)(input_1)
lstm_2 = tf.keras.layers.LSTM(6)(input_2)
lstm_3 = tf.keras.layers.LSTM(5)(input_3)

In [ ]:
lstm_1

In [ ]:
lstm_2

In [ ]:
lstm_3

In [ ]:
#Concatenate LSTM layers output
cat = tf.keras.layers.concatenate([lstm_1, lstm_2, lstm_3])

In [ ]:
cat

In [ ]:
#Output Layer
op = tf.keras.layers.Dense(1)(cat)

In [ ]:
op

In [ ]:
#Build Non-Sequential Model
model_multi = tf.keras.Model([input_1, input_2, input_3], #3 Inputs
                             op) #Output

In [ ]:
#compile model 
model_multi.compile(optimizer='adam', loss='mae')

In [ ]:
model_multi.summary()

##### Model Training

We need to feed 3 inputs. Each input will be 3 dimensional

In [ ]:
np.array(train_x_multi[0]).shape

In [ ]:
#Build data for training
train_x_multi_1 = np.reshape(np.array(train_x_multi[0]), (len(train_x_multi[0]),len(train_x_multi[0][1]),1 ))
train_x_multi_2 = np.reshape(np.array(train_x_multi[1]), (len(train_x_multi[1]),len(train_x_multi[1][1]),1 ))
train_x_multi_3 = np.reshape(np.array(train_x_multi[2]), (len(train_x_multi[2]),len(train_x_multi[2][1]),1 ))

In [ ]:
train_x_multi_1.shape

In [ ]:
train_x_multi_2.shape

In [ ]:
train_x_multi_3.shape

In [ ]:
#Build data for test
test_x_multi_1 = np.reshape(np.array(test_x_multi[0]), (len(test_x_multi[0]),len(test_x_multi[0][1]),1 ))
test_x_multi_2 = np.reshape(np.array(test_x_multi[1]), (len(test_x_multi[1]),len(test_x_multi[1][1]),1 ))
test_x_multi_3 = np.reshape(np.array(test_x_multi[2]), (len(test_x_multi[2]),len(test_x_multi[2][1]),1 ))

In [ ]:
test_x_multi_1.shape

In [ ]:
#Model training
model_multi.fit([train_x_multi_1, train_x_multi_2, train_x_multi_3], np.array(train_y_multi), 
                validation_data=([test_x_multi_1, test_x_multi_2, test_x_multi_3], np.array(test_y_multi)), 
                epochs=5, 
                batch_size=256)

Visualize Model Prediction

In [ ]:
#Pick a test example
exp_num = np.random.randint(0, len(test_y_multi))

In [ ]:
exp_num

In [ ]:
np.array(test_x_multi[0][exp_num]).shape

In [ ]:
np.expand_dims(np.array(test_x_multi[0][exp_num]), axis=0).shape

In [ ]:
#Pick a test example
exp_num = np.random.randint(0, len(test_y_multi))

#Prepare 3 batch inputs - each 3 dimensional
in_1 = np.expand_dims(np.array(test_x_multi[0][exp_num]), axis=0)
in_2 = np.expand_dims(np.array(test_x_multi[1][exp_num]), axis=0)
in_3 = np.expand_dims(np.array(test_x_multi[2][exp_num]), axis=0)

#print(in_1.shape, in_2.shape)
#Make prediction
prediction = model_multi.predict([in_1, in_2, in_3])

#Visualize
show_example(in_1[0], test_y_multi[exp_num], prediction[0])

**Using Single LSTM** with same time window for multiple time series

In [ ]:
train_x_multi, train_y_multi = prepare_xy_multi(norm_train_df, window_size=(20,20,20))
test_x_multi, test_y_multi = prepare_xy_multi(norm_test_df, window_size=(20,20,20))

In [ ]:
len(train_x_multi)

In [ ]:
np.array(train_x_multi[2]).shape

In [ ]:
train_x = np.concatenate([np.reshape(train_x_multi[0], (-1, 20,1)), 
                          np.reshape(train_x_multi[1], (-1, 20,1)), 
                          np.reshape(train_x_multi[2], (-1, 20,1))], axis=2) 

In [ ]:
train_x.shape

In [ ]:
test_x = np.concatenate([np.reshape(test_x_multi[0], (-1, 20,1)), 
                          np.reshape(test_x_multi[1], (-1, 20,1)), 
                          np.reshape(test_x_multi[2], (-1, 20,1))], axis=2) 

In [ ]:
test_x.shape

In [ ]:
tf.keras.backend.clear_session()
model1 = tf.keras.Sequential()

In [ ]:
model1.add(tf.keras.layers.LSTM(8, input_shape=(20,3)))

In [ ]:
model1.output

In [ ]:
model1.add(tf.keras.layers.Dense(1))

In [ ]:
model1.compile(optimizer='adam', loss='mae')

In [ ]:
model1.summary()

In [ ]:
model1.fit(train_x, np.array(train_y_multi), epochs=5, batch_size=200)

In [ ]:
#Pick a test example
exp_num = np.random.randint(0, test_x.shape[0])

#Prepare 3 batch inputs - each 3 dimensional
in_1 = np.expand_dims(test_x[exp_num], axis=0)

#Make prediction
prediction = model1.predict(in_1)

#Visualize
#show_example(in_1[0], test_y_multi[exp_num], prediction[0])
prediction